In [3]:
## Code that shifts data 15 days and adds signals then predicts that way

In [4]:
import pandas as pd
import numpy as np
import requests
import json
import hvplot.pandas
from dotenv import load_dotenv
from datetime import date
import os
from scipy import stats
from numpy.lib.stride_tricks import as_strided
from numpy.lib import pad
#import pad
import matplotlib.pyplot as plt
%matplotlib inline

from pathlib import Path

In [5]:
# Initial imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import model_from_json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [7]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [8]:
def rolling_spearman(seqa, seqb, window):
    stridea = seqa.values.strides[0]
    ssa = as_strided(seqa, shape=[len(seqa) - window + 1, window], strides=[stridea, stridea])
    strideb = seqa.values.strides[0]
    ssb = as_strided(seqb, shape=[len(seqb) - window + 1, window], strides =[strideb, strideb])
    ar = pd.DataFrame(ssa)
    br = pd.DataFrame(ssb)
    ar = ar.rank(1)
    br = br.rank(1)
    corrs = ar.corrwith(br, 1)
    return pad(corrs, (window - 1, 0), 'constant', constant_values=np.nan)

def use_csvs(ticker):

    data = pd.read_csv("../FilesExportIndividualStockDFs_Big/"+ticker+"_combined_df.csv", index_col='Date', parse_dates=True)
#     path = Path('../FilesExport_Finished/'+ticker+'_finished_df.pkl')
#     data_import = load_obj(path)
#     data = data_import['dataFrame'].copy()

    return data.iloc[0:-1]



def bollinger_bands(dataframe,period=20):
    data = dataframe.copy()
    data['middle_band'] = data[['adjClose']].rolling(window=period).mean()
    data[str(period)+'_day_stdev'] = data[['adjClose']].rolling(window=period).std()
    data['upper_band'] = data['middle_band']+2*data[str(period)+'_day_stdev']
    data['lower_band'] = data['middle_band'] - 2*data[str(period)+'_day_stdev']
    data['spread'] = data['upper_band'] + data['lower_band']
    data['change_in_spread'] = data['spread']/data['spread'].shift(1)-1
    data[str(period)+"_return"] = data['adjClose']/data['adjClose'].shift(period)-1
    data['bollinger_signal'] = data['change_in_spread'].rank(ascending=False, pct=True)
    data.dropna()

    return data

def dema(dataframe, period1=10, period2=20):
    data = dataframe.copy()
    data[str(period1)+'ema1'] = dataframe[['adjClose']].ewm(span=period1, adjust=False).mean()
    data[str(period1)+'ema2'] = data[str(period1)+'ema1'].ewm(span=period1, adjust=False).mean()
    data[str(period1)+'dema'] = 2*data[str(period1)+'ema1'] - data[str(period1)+'ema2']
    data[str(period2)+'ema1'] = data[['adjClose']].ewm(span=period2, adjust=False).mean()
    data[str(period2)+'ema2'] = data[str(period2)+'ema1'].ewm(span=period2, adjust=False).mean()
    data[str(period2)+'dema'] = 2*data[str(period2)+'ema1'] - data[str(period2)+'ema2']
    data[str(period1)+"_return"] = data['adjClose']/data['adjClose'].shift(period1)-1
    data['spread'] = data[str(period1)+'dema'] - data[str(period2)+'dema']
    data['dema_signal'] = data['spread'].rank(ascending=True, pct=True)
    data = data.dropna()
    return data

def price_momentum(dataframe, smoothing1=0.0571, smoothing2=0.1, periods1=15, periods2=10):
    data = dataframe.copy()
    data['smoothing_factor'] = smoothing1
    data[str(periods1)+"average"] = data['changeOverTime'].rolling(window=periods1).mean()
    smoothing_factor_list = [data.iloc[periods1][str(periods1)+"average"]]
    data = data.dropna()
    i=1
    j=0
    while i < len(data[str(periods1)+"average"]):
        smoothing_factor = data.iloc[i]['changeOverTime']*data.iloc[i]['smoothing_factor'] + smoothing_factor_list[j]*(1-data.iloc[i]['smoothing_factor'])
        smoothing_factor_list.append(smoothing_factor)
        j+=1
        i+=1
    data['35d_custom_smoothing'] = smoothing_factor_list
    data['35d_custom_10'] = data['35d_custom_smoothing']*10
    data['smoothing_factor2'] = smoothing2
    data[str(periods2)+"average"] = data['35d_custom_10'].rolling(window=periods2).mean()
    data = data.dropna()
    smoothing_factor_list2 = [data.iloc[0][str(periods2)+"average"]]
    i=1
    j=0
    while i < len(data[str(periods2)+"average"]):
        smoothing_factor = (data.iloc[i]['35d_custom_10'] - smoothing_factor_list2[j])*data.iloc[i]['smoothing_factor2'] + smoothing_factor_list2[j]
        smoothing_factor_list2.append(smoothing_factor)
        j+=1
        i+=1
    data[str(periods2)+'d_custom_smoothing'] = smoothing_factor_list2
    data[str(periods2)+"_return"] = data['adjClose']/data['adjClose'].shift(periods2)-1
#         data['signal'] = np.where(data[str(periods2)+'d_custom_smoothing'] > data[str(periods2)+'d_custom_smoothing'].shift(1), 1.0, 0.0)
#         data = data.rename(columns={'signal':'price_mo'})
    return data

def get_ichimoku_cloud(dataframe, period1=4, period2=8, period3=15):

    #TODO generate signal, ichimoku works better in current market regime with shorter periods, being able to respond faster to events than a traditional version
    # The conversion crossing the base would be the signal

    data = dataframe.copy()
    data['conversion_line'] = data[['adjClose']].rolling(window=period1).mean()
    data['base_line'] = data[['adjClose']].rolling(window=period2).mean()
    data['senkou_spanA_line'] = (data['conversion_line']+data['base_line'])/2
    data['senkou_spanB_line'] = data[['adjClose']].rolling(window=period3).mean()
    data['lagging_span'] = data['adjClose'].shift(period2)
    data = data.dropna()

    return data

def accumulation_distribution_line(dataframe):

    ##TODO define periodicity and pass as arguments, use the mean as the signal generator, -1 is buy and and 1 is sell
    ##TODO need to add ability to ignore a -1 during a range of 1s

    data = dataframe.copy()
    data['money_flow_mult'] = round(((data['adjClose'] - data['low']) - (data['high'] - data['adjClose']))/(data['high'] - data['low']),2)
    data = data.dropna()
    data['money_flow_volume'] = data['money_flow_mult']*data['volume']
    money_flow_multiplier_list = list(data['money_flow_volume'].values)
    adl = [money_flow_multiplier_list[0]]
    i = 1
    while i < len(money_flow_multiplier_list):
        a_d_indicator = adl[i-1]+money_flow_multiplier_list[i]
        adl.append(a_d_indicator)
        i+=1
    data['adl'] = adl
    data['adl_change'] = data['adl']/data['adl'].shift(1)-1
    negative_change_count = [0]*9
    i = 0
    counter = 0
    while i < len(data)-9:
        j=0
        while j < 9:
            if data.iloc[j+i]['adl_change'] <0:
                counter+=1
            if j %19 == 0:
                negative_change_count.append(counter)
                counter = 0
            j+=1
        i+=1
    data['negative_change_counter'] = negative_change_count
    data['9_day_return'] = data['adjClose']/data['adjClose'].shift(9)-1
    data['adl_signal'] = rolling_spearman(data['adl'], data['9_day_return'], 9)

    return data

def rsi(dataframe, periods=14):
    data = dataframe.copy()
    data['gains'] = np.where(data['changeOverTime']>0, data['changeOverTime'], 0)
    data['losses'] = np.where(data['changeOverTime']<0, np.absolute(data['changeOverTime']), 0)
    data['average_gain'] = data['gains'].rolling(window=periods).mean()
    data['average_loss'] = data['losses'].rolling(window=periods).mean()
    data['rs'] = data['average_gain']/data['average_loss']
    data['rsi'] = (100 - 100/(1+data['rs']))
    data['rsi_signal'] = data['rsi'].rank(ascending=True, pct=True)

    return data

def get_all_indicators(ticker):
    dataframe = use_csvs(ticker)
    bb = bollinger_bands(dataframe)
    DEMA = dema(dataframe)
    ADL = accumulation_distribution_line(dataframe)
    RSI = rsi(dataframe)

    dataframe['bollinger_signal'] = bb['bollinger_signal']
    dataframe['dema_signal'] = DEMA['dema_signal']
    dataframe['adl_signal'] = ADL['adl_signal']
    dataframe['rsi_signal'] = RSI['rsi_signal']
    dataframe = dataframe.dropna()
    return dataframe

In [9]:
def shift_dataset(dataframe, period=15):
    data= dataframe.copy()
    data = data.dropna()
    shifted_dataframe = data.drop(['close', 'adjClose'], axis=1).shift(-period)
    shifted_dataframe = shifted_dataframe.dropna()
    data_close = data[['close']].iloc[period:]
    shifted_dataframe.loc[:,'close'] = data_close.values
    
    return shifted_dataframe

In [ ]:
def do_all()